In [83]:
from urllib.request import urlretrieve
import requests
from bs4 import BeautifulSoup
import time
import shutil, os.path

In [119]:
pages = []
for i in range(1, 6):
    url = ('https://repository.tudelft.nl/islandora/search?page=' + str(i) + 
    '&collection=education&f%5B0%5D=mods_genre_s%3A%22master%5C%20thesis%22')
    pages.append(url)

In [123]:
i = 0
j = 0
for item in pages:
    page = requests.get(item)
    soup = BeautifulSoup(page.text, 'html.parser')
    thesis_list = soup.find(class_='islandora-basic-collection-list')
    thesis_list_items = thesis_list.find_all('a')
    print(item)
    for thesis_name in thesis_list_items:
        url = thesis_name.get('href')
        uuid = url[25:-21]
        download = 'https://repository.tudelft.nl/islandora/object/uuid:' + uuid + '/datastream/OBJ/download'     
        name = 'data/tudelft_repo/TUD_' + uuid + '.pdf' 
        if os.path.exists(name):
            print('File exists:', name)
            i = i + 1
            continue
        r = requests.get(download, stream=True)
        if r.status_code == 200:
            with open(name, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
            print('Downloaded:', name)
            j = j + 1
        if r.status_code != 200:
            ('Download failed, status', r.status_code, 'link', download)

        time.sleep(1)
        
print(i, 'files already existed,', j, 'files downloaded')

https://repository.tudelft.nl/islandora/search?page=1&collection=education&f%5B0%5D=mods_genre_s%3A%22master%5C%20thesis%22
File exists: data/tudelft_repo/TUD_1575733a-8813-4c5d-b916-0eb23ac477f9.pdf
File exists: data/tudelft_repo/TUD_05957f39-d946-4956-be68-52f5675e100d.pdf
File exists: data/tudelft_repo/TUD_e675fadc-fd8d-4ced-b59e-45b6faba6fdd.pdf
File exists: data/tudelft_repo/TUD_c9492ef8-9254-4ba4-83b3-8b0ad46149c5.pdf
File exists: data/tudelft_repo/TUD_6e274095-9920-4d20-9e11-d5b76363e709.pdf
File exists: data/tudelft_repo/TUD_b2308402-21ed-41a2-92b7-7edd7175dca7.pdf
File exists: data/tudelft_repo/TUD_c46eca4b-eaf5-461a-9be2-dd6b3b1b44e0.pdf
File exists: data/tudelft_repo/TUD_ca8da1e3-94c0-438b-9281-ee4a76cb7129.pdf
File exists: data/tudelft_repo/TUD_15227443-568e-4bd8-af88-a165c4776f11.pdf
File exists: data/tudelft_repo/TUD_cff45740-9100-4a10-9d49-41da874146d5.pdf
https://repository.tudelft.nl/islandora/search?page=2&collection=education&f%5B0%5D=mods_genre_s%3A%22master%5C%20th

# OAI Extraction

In [199]:
from urllib.request import urlretrieve
from bs4 import BeautifulSoup 
from sickle import Sickle
import shutil, os.path, pickle, time, requests
from collections import Counter

In [2]:
sickle = Sickle('http://oai.tudelft.nl/ir')

In [64]:
records = sickle.ListIdentifiers(metadataPrefix='oai_dc')

In [147]:
records = sickle.ListRecords(metadataPrefix='oai_dc', ignore_deleted=True)

In [306]:
items = {}
records = sickle.ListRecords( **{'metadataPrefix': 'oai_dc', 'from': '2016-01-01', 'ignore_deleted': 'True'})
for x in range(20000):
    item = records.next()
    uuid = item.metadata['identifier'][0][32:]
    items[uuid] = item.metadata
    
print(len(items))

18428


In [345]:
x = 0
types = []
items = {}

records = sickle.ListRecords( **{'metadataPrefix': 'oai_dc', 'ignore_deleted': 'True'})
for r in records:
    types.append(r.metadata['type'][0])
    uuid = ''
    uuid = r.metadata['identifier'][0][32:]
    items[uuid] = r.metadata
    x = x + 1

print('items', len(items), 'types', len(types), 'x', x)
Counter(types) 

items 57771 types 62105 x 62105


Counter({'Text': 1,
         'abstract': 155,
         'bachelor thesis': 1128,
         'book': 1926,
         'book chapter': 525,
         'conference paper': 10767,
         'contribution to periodical': 29,
         'doctoral thesis': 8331,
         'journal article': 14483,
         'lecture notes': 489,
         'master thesis': 16201,
         'other': 77,
         'patent': 544,
         'public lecture': 1228,
         'report': 5797,
         'review': 46,
         'student report': 300,
         'working paper': 78})

In [ ]:
with open('tud_metadata.pickle', 'wb') as handle:
    pickle.dump(items, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [403]:
with open('tud_metadata.pickle', 'rb') as handle:
    items = pickle.load(handle)  

In [ ]:
failed = []
for uuid in items.keys():
    download = 'https://repository.tudelft.nl/islandora/object/uuid:' + uuid + '/datastream/OBJ/download'     
    name = 'data/tudelft_repo/TUD_' + uuid + '.pdf' 
    if os.path.exists(name):
        print('File exists:', name)
        print('.', end = '')
        continue
    r = requests.get(download, stream=True)
    if r.status_code == 200:
        with open(name, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)
        print('Downloaded:', name)
    if r.status_code != 200:
        print('Download failed, status', r.status_code, 'link', download)
        failed.append(uuid)
    time.sleep(3)

In [371]:
downloaded = {}
for dirpath, dirs, files in os.walk('data/tudelft_repo/'):
    for file in files: 
        path = os.path.join(dirpath, file)
        uuid = path.split('_')[-1].split('.')[0]
        downloaded[uuid] = items[uuid]

In [410]:
import re
for key in list(downloaded.keys()):
    if downloaded[key]['type'][0] == 'master thesis':
        try:
            if any('Ferreira' in item for item in downloaded[key]['contributor']):
                print(downloaded[key]['title'])
                print(downloaded[key]['subject'])
#                 text = downloaded[key]['description']
#                 print(re.sub('<.*?>', ' ', text[0]))
                print('')
        except:
            pass

['Measurement and modelling of earthing impedance in ships (DC-30MHz)']
['measurement', 'modelling', 'earthing wire', 'external inductance', 'internal inductance']

['Airfoil Optimization for vertical axis wind turbines']
['vertical axis wind turbine']

['A Hybrid Renewable Energy System for a Rural Area in Africa']
['renewable energy', 'sustainability', 'Ikem', 'load demand', 'hydropower', 'photovoltaic', 'biomass combustion', 'HRES', 'Hybrid Renewable Energy System', 'rural electrification']

['Sensitivity analysis of residential building simulations: Model choice for specific applications and critical parameters']
['TRNSYS', 'simulation', 'influential behavior']

['Aerodynamic response of a twodimensional vertical axis wind turbine to turbulence']
['Dynamic Analysis of a Floating Vertical Axis Wind Turbine using the Actuator Cylinder Flow Theory']
['Vertical Axis Wind Turbines (VAWTs)', 'AC Flow Theory', 'Fully coupled dynamic analysis', 'Code-to-code comparison', 'HAWC2', 'SIMO-RIF

In [405]:
downloaded[key]['publisher']


['Vlaams Steunpunt Ruimte en Wonen']

# Delete Large Files

In [348]:
import os
from matplotlib import pyplot as plt

target_size = 100 * 1048514
large = []

for dirpath, dirs, files in os.walk('data/tudelft_repo/'):
    for file in files: 
        path = os.path.join(dirpath, file)
        if os.stat(path).st_size > target_size:
            os.remove(path)
            large.append(file)

In [271]:
with open('large_deleted.pickle', 'wb') as handle:
    pickle.dump(large, handle, protocol=pickle.HIGHEST_PROTOCOL)